In [2]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download the stopwords list (only once)
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

##Non-Open ended question

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util


# Define keywords or attribute phrases

# Example description



# Load a pre-trained sentence transformer model

# Function to score a description
def club_score(description, attributes):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    scores = {}
    desc_embedding = model.encode(description, convert_to_tensor=True)

    for attr, phrases in attributes.items():
        attr_embeddings = model.encode(phrases, convert_to_tensor=True)
        similarity = util.cos_sim(desc_embedding, attr_embeddings)
        scores[attr] = similarity.mean().item()  # Average similarity score

    return scores



# Assess the description
def match_clubs(club_descriptions,user_attributes):
  matching_clubs = {}
  attributes = { #WARNING: Words are subjected to change
    "teamwork": ["networking", "collaboration", "partnership", "coordinate","support"], # "cooperation", "team", "collaboration", "group work", "mutual support", "collective effort", "synergy", "shared responsibility", "coordination", "joint effort", "supportive environment", "team spirit", "interdependence", "togetherness", "cooperation across teams", "team-based", "group dynamics"
    "community_service": ["awareness","community", "volunteer", "giving back", "charity","society"], # "charity", "social impact", "community engagement","helping others", "civic responsibility", "social responsibility", "philanthropy", "mission trips", "voluntary service", "empowerment", "public service", "nonprofit work", "community involvement", "advocacy"
    "leadership": ["leader", "excel", "succeed", "evolve","future",], # "leadership", "leader", "inspire", "mentor", "guide", "decision-making", "visionary","responsibility", "motivator", "manager", "influence", "coach", "authority", "empower", "director", "organizer", "strategic thinking", "leadership development", "leading by example"
    "eagerness_to_learn": ["interest","passion","curious", "explore","develop"], #"learning", "curiosity", "knowledge-seeking", "growth mindset", "self-improvement", "development", "education", "exploration", "adaptability", "open-mindedness", "lifelong learning", "innovation","self-driven", "intellectual curiosity", "new experiences", "skill-building", "challenging oneself","personal growth", "exploration of new ideas", "knowledge expansion"
    "critical_thinking": ["analysis", "problem-solving", "evaluate", "strategy","critical thinking"] # "problem-solving", "analysis", "critical thinking", "reasoning", "evaluation", "decision-making", "strategy","logic", "objectivity", "judgment", "reflection", "inquiry", "debate", "data analysis", "questioning assumptions","cognitive skills", "thoughtful consideration", "synthesis", "problem resolution", "insight"
}

  for i in range(len(club_descriptions)):
    categories = ["teamwork", "community_service", "leadership", "eagerness_to_learn", "critical_thinking"]
    scores = dict(sorted(club_score(club_descriptions[i], attributes).items(), key=lambda item: item[1],reverse = True))
    for attr in range(len(user_attributes)):
      if (len(matching_clubs) < 6 and user_attributes[attr]) and scores[categories[attr]] > 0.15:
        matching_clubs[i] = scores[categories[attr]]

  sorted_dict = dict(sorted(matching_clubs.items(), key=lambda item: item[1], reverse = True))
  return list(sorted_dict.keys())



##Open_Ended Question

In [66]:
import re
import numpy as np



#filtering sentence to make similarity scores more relevent
def filter_sentence(sentence):

  # Step 1: Tokenize the sentence into words
  words = word_tokenize(sentence)

  # Step 2: Get the list of stopwords in English
  stop_words = set(stopwords.words('english'))

  # Step 3: Remove stopwords from the sentence
  filtered_words = [word for word in words if word.lower() not in stop_words]

  # Step 4: Reconstruct the sentence without stopwords
  filtered_sentence = ' '.join(filtered_words)

  # Output the result
  return filtered_sentence

#finding relevent clubs
#TODO: Might need to split a group of sentences into individual sentences and check mean of similarity score for each sentence
def interest_similarity(club_descriptions, user_input):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  unfiltered_clubs = {}
  for i in range(len(club_descriptions)):  #TODO: needs more changes related sorting
    user_embedding = model.encode(user_input, convert_to_tensor=True)
    club_embedding = model.encode(club_descriptions[i], convert_to_tensor=True)
    cosine_similarity = util.cos_sim(user_embedding, club_embedding)
    if len(unfiltered_clubs) < 6 and cosine_similarity > 0.2:
      unfiltered_clubs[i] = cosine_similarity

  sorted_dict = dict(sorted(unfiltered_clubs.items(), key=lambda item: item[1],reverse = True))
  return list(sorted_dict.keys())







#Final Implementation

In [67]:

def indicesOfMatchingClubs(club_descriptions, teamwork,community_service,leadership,learning,critical_thinking,hobbies,club_wants):
  questions = {"teamwork": teamwork,
          "community_service": community_service,
          "leadership": leadership,
          "learning": learning,
          "critical_thinking": critical_thinking,
          "hobbies": hobbies,
          "club_wants": club_wants
          }
  user_input = [questions["hobbies"] + ". " + questions["club_wants"]]
  user_attributes = [questions["teamwork"],questions["community_service"],questions["leadership"],questions["learning"],questions["critical_thinking"]]
  return list(np.unique(np.array((interest_similarity(club_descriptions, user_input) + match_clubs(club_descriptions, user_attributes)))))

def listOfMatchingClubs(club_titles,club_descriptions, teamwork,community_service,leadership,learning,critical_thinking,hobbies,club_wants):
  finalClubs = {}
  for i in indicesOfMatchingClubs(club_descriptions, teamwork,community_service,leadership,learning,critical_thinking,hobbies,club_wants):
    finalClubs[club_titles[i]] = club_descriptions[i]
  if finalClubs == {}:
    finalClubs = {"No matching clubs found":""}
  return finalClubs


#Testing the model

In [68]:
#for reference purposes only
questions = {"teamwork": False,
            "community_service": False,
            "leadership": False,
            "learning": False,
            "critical_thinking": False,
            "hobbies": "I am into movies",
            "club_wants": ""
            }

#actual input that varies the output
club_descriptions = [
    "Aa Dekhen Zara (ADZ) is an annual South Asian Dance competition hosted by the students of the University of Wisconsin-Madison. We host talented Bollywood-Fusion dance teams.",
    "The Alexander Hamilton Society (AHS) is a non-partisan organization focused on foreign policy and national security international relations.",
    "The American Medical College of Pharmacy (AMCP) provides opportunities to learn about managed care pharmacy, improving patient outcomes through clinical and business knowledge.",
    "Acacia is a social fraternity dedicated to human service. We are dedicated to bettering the community we are a part of and sharing a brotherhood we will remember for the rest of our lives. We are a fraternity that is focused on the strength of brotherhood. By selecting key members of the Madison community, we build a brotherhood dedicated to human service for both the community and ourselves. These members are selected through the IFC-regulated RUSH process in which members are first bid followed by a pledge process. We are a non-hazing fraternity built on the idea that disrespect to any possible future member or person is strictly against our goals and morals. All leaders of the Acacia Fraternity are elected by the active members and must be students of the University of Wisconsin-Madison.  ",
"The Health Entreprenuers and Leaders is a prehealth student organization established at the University of Wisconsin-Madison in 2020. As prehealth students, we identified the gaps and disparities in our community and the medical field. HEAL is our legacy to become the change that goes beyond addressing the issue but pushes forward to act upon them at the core of the issues. We understand Health care policy and insurance are crucial factors that comes to healthcare affordability and accessibility, which HEAL has a focus to address. Beyond this, the cornerstone of HEAL is to address problems in healthcare at all levels, including but not limited to the federal, state, hospital administration, healthcare providers/professionals, patients, etc. HEAL is structured to be a center for all pre-health students to attain resources and mentorship to grow and develop in their path to pursuing and entering the health field. HEAL understands the benefits and advocates for the diversity of people, backgrounds, and minds to improve the future of medicine."
]


club_titles = ["Aa Dekhen Zara", "The Alexander Hamilton Society", "The American Medical College of Pharmacy", "Acacia", "The Health Entreprenuers and Leaders"]
event_descriptions = ["event1", "event2", "event3","event4","event5"]



import csv

with open('/content/organizations_data.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    organizations_data = [row for row in csv_reader][0:5]


# Extract only the descriptions from the organizations data
club_descriptions = [row['description'] for row in organizations_data]
club_title = [row['name'] for row in organizations_data]
print(listOfMatchingClubs(club_title,club_descriptions,True,False,True,False,True,"i like going out and playing with computers. I also enjoy coding","I want a social environment with comnputers"))

{'Acacia Fraternity - Lamedth Chapter': '', 'Academy of Managed Care Pharmacy Student Chapter at UW Madison': 'AMCP is a national professional organization of pharmacists and other health care providers that aims to improve patient outcomes through the merger of both clinical and business knowledge. The student chapter allows those interested to learn more about this subset of pharmacy. The student chapter seeks to provide students with a greater understanding of the field of managed care pharmacy. Students are provided multiple opportunities to strengthen their knowledge of this area, such as engaging with speakers who are current professionals in the field as well as participating in a national pharmacy and therapeutics competition.'}
